In [1]:
!pip install pandas_datareader
from pandas_datareader import data
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import talib as ta
import warnings
warnings.simplefilter('ignore')

!pip install yahoo_finance_api2
import sys
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError

In [2]:
def stock_data(stock_code,frequency=1,weekly=True):
    my_share = share.Share(stock_code)
    symbol_data = None
    
    if weekly == True:
        symbol_data = my_share.get_historical(share.PERIOD_TYPE_YEAR,
                                              1,
                                              share.FREQUENCY_TYPE_DAY,
                                             1)
        
    elif weekly == False:
        symbol_data = my_share.get_historical(share.PERIOD_TYPE_YEAR,
                                              1,
                                              share.FREQUENCY_TYPE_HOUR,
                                             1)

    df = pd.DataFrame(symbol_data)
    date_agg=lambda x:datetime.datetime.utcfromtimestamp(x/1000)
    df['timestamp'] = df['timestamp'].apply(date_agg)
    if weekly ==True:
        to_day=lambda x:datetime.datetime.strftime(x,'%A')
        df["day"]=df['timestamp'].apply(to_day)
        df_new=pd.DataFrame()
        for i in range(len(df.loc[df["day"]=="Tuesday"])):
            if df.iloc[df.loc[df["day"]=="Tuesday"].index[i]-1]["day"]=="Monday":
                df_=df.iloc[df.loc[df["day"]=="Tuesday"].index[i]-1]
                df_new=pd.concat([df_new,df_],axis=1)
            else:
                df_=df.iloc[df.loc[df["day"]=="Tuesday"].index[i]]
                df_new=pd.concat([df_new,df_],axis=1)
        df_new=df_new.T
    else:
        df_new=df
    
    
    return df_new

In [3]:
stock_data("ACN",frequency=1,weekly=True)

AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

In [4]:
stock_data("ACN",1,False)

AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

In [5]:
weekly_df.loc[weekly_df["day"]=="Tuesday"].index[i]

NameError: name 'weekly_df' is not defined

In [6]:
weekly_df["day"]=weekly_df['timestamp'].apply(to_day)

NameError: name 'weekly_df' is not defined

In [7]:
weekly_df.loc[weekly_df["day"]=="Tuesday"].index[]

SyntaxError: invalid syntax (<ipython-input-7-edfb7dd099cc>, line 1)

In [8]:
pd.set_option('display.max_rows',1000)
weekly_df['RSI'] = ta.RSI(weekly_df['close'],timeperiod=14)

weekly_df

NameError: name 'weekly_df' is not defined

In [9]:
ETF=['ACN','DIA','SPY','QQQ','IBB','XLV','IWM','EEM','EFA','XLP','XLY','ITB','XLU','XLF',
     'VGT','VT','FDN','IWO','IWN','IYF','XLK','XOP','USMV','BAB','GLD',
    'VNQ','SCHH','IYR','XLRE','AGG','BND','LQD','VCSH','VCIT','JNK']

In [10]:

    def performance_check(data,macd_flag,code):
        max_=np.max(macd_flag)
        performance_list=[]
        if max_ %2 ==0:
            for i in range(1,max_,2):
                performance=data[macd_flag.index(i+1)]/data[macd_flag.index(i)]
                profit=data[macd_flag.index(i+1)]-data[macd_flag.index(i)]
                buy_date=data.index[macd_flag.index(i)]
                sell_date=data.index[macd_flag.index(i+1)]
                performance_dict={"code":code,"peformance":performance,"profit":profit,
                                         "buy_date":buy_date,"sell_date":sell_date,}
                performance_list.append(performance_dict)

        else: 
            max_-=1
            for i in range(1,max_,2):
                performance=data[macd_flag.index(i+1)]/data[macd_flag.index(i)]
                profit=data[macd_flag.index(i+1)]-data[macd_flag.index(i)]
                buy_date=data.index[macd_flag.index(i)]
                sell_date=data.index[macd_flag.index(i+1)]
                performance_dict={"code":code,"peformance":performance,"profit":profit,
                                         "buy_date":buy_date,"sell_date":sell_date}
                performance_list.append(performance_dict)
        df=pd.DataFrame(performance_list)
        return df

In [11]:
df_performance=pd.DataFrame()
for i in range(1):
    
    df = stock_data(ETF[i],1,False)
    df = df.loc[df['close'].notnull()]
    code=ETF[i]
    date = df['timestamp']
    close = df['close']
    
    
    df['macd'],df['macdsignal'],df['macdhist'] = ta.MACD(close,fastperiod=12,slowperiod=26,signalperiod=9)
    df['RSI'] = ta.RSI(close,timeperiod=14)
    #移動平均なので、最初のspan日だけ欠損値になる。
    #それを防ぐために、ループを掛ける前に修正を行う
    rsi_list=df["RSI"][df["macd"].notna()].reset_index(drop=True)
    macd_list,signal_list=df['macd'][df["macd"].notna()].reset_index(drop=True),df['macdsignal'][df["macd"].notna()].reset_index(drop=True)
    close_revised=close[df["macd"].notna()].reset_index(drop=True)

    rsi_flag = 0
    macd_flag = []
    buy_flag=0
    sell_flag=0
    for i in range(len(rsi_list)):

        #simulate when to buy
        if (buy_flag % 2==0) or (buy_flag==0):

            if rsi_list[i] <= 20:
                rsi_flag = 1
            if (signal_list[i]-macd_list[i]<=0) & (rsi_flag == 1):

                buy_flag+=1
                rsi_flag-=1
                macd_flag.append(buy_flag)
            else:macd_flag.append(buy_flag)
        #simulate when to sell
        elif buy_flag %2 !=0 :
            if rsi_list[i]>=70:
                sell_flag=1
            if (sell_flag==1) & (signal_list[i]-macd_list[i]>=0):
                buy_flag+=1
                macd_flag.append(buy_flag)

            else: macd_flag.append(buy_flag)

    df_i=performance_check(close_revised,macd_flag,code)
    print(df_i)
    df_performance=pd.concat([df_performance,df_i],axis=0)
    print(df_performance)
    

AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

In [12]:
df_performance.describe()

ValueError: Cannot describe a DataFrame without columns

In [13]:
df_performance

""


In [14]:
rsi_list

NameError: name 'rsi_list' is not defined

In [96]:
    
df = stock_data(ETF[0])
df = df.loc[df['close'].notnull()]
code=ETF[0]
date = df['timestamp']
close = df['close']

df['macd'],df['macdsignal'],df['macdhist'] = ta.MACD(close,fastperiod=12,slowperiod=26,signalperiod=9)
df['RSI'] = ta.RSI(close,timeperiod=14)
#移動平均なので、最初のspan日だけ欠損値になる。
#それを防ぐために、ループを掛ける前に修正を行う
rsi_list=df["RSI"][df["RSI"].notna()]
macd_list,signal_list=df['macd'][df["RSI"].notna()],df['macdsignal'][df["RSI"].notna()]
close_revised=close[df["RSI"].notna()]

df.isnull().sum()

timestamp      0
open           0
high           0
low            0
close          0
volume         0
macd          33
macdsignal    33
macdhist      33
RSI           14
dtype: int64

In [97]:
df.loc[df['macd'].isnull()]

,timestamp,open,high,low,close,volume,macd,macdsignal,macdhist,RSI
0,2020-02-10 14:30:00,210.190002,212.960007,210.190002,212.804306,0.0,NaN,NaN,NaN,NaN
1,2020-02-10 15:30:00,212.770004,212.970001,212.119995,212.410004,121344.0,NaN,NaN,NaN,NaN
2,2020-02-10 16:30:00,212.380005,212.660004,212.179993,212.470001,74392.0,NaN,NaN,NaN,NaN
3,2020-02-10 17:30:00,212.479996,212.520004,212.169098,212.190002,78717.0,NaN,NaN,NaN,NaN
4,2020-02-10 18:30:00,212.210007,212.240005,211.669998,211.699997,90550.0,NaN,NaN,NaN,NaN
5,2020-02-10 19:30:00,211.679993,212.059998,211.580002,211.921005,179191.0,NaN,NaN,NaN,NaN
6,2020-02-10 20:30:00,211.929993,212.570007,211.830002,212.550003,233465.0,NaN,NaN,NaN,NaN
7,2020-02-10 21:00:00,212.550000,212.550000,212.550000,212.550000,0.0,NaN,NaN,NaN,NaN
8,2020-02-11 14:30:00,213.380005,213.979996,212.470001,213.300003,242208.0,NaN,NaN,NaN,NaN
9,2020-02-11 15:30:00,213.330002,213.509995,212.570007,212.674606,164265.0,NaN,NaN,NaN,NaN


In [93]:
df.loc[df['close'].isnull()]

,timestamp,open,high,low,close,volume,macd,macdsignal,macdhist,RSI
1972,2020-11-27 17:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2158,2020-12-24 17:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
rsi_list[33]

48.2718417794956